In [ ]:
import sys
import os
sys.path.append(r"C:\Users\Abdessamad\Desktop\MLOpsClassificationTexteV2\src")
import data.loader as loader
import training.train as train
import pandas as pd
import random
from models.factory import get_model
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import warnings
import mlflow
import mlflow.sklearn
from typing import List


In [2]:
warnings.filterwarnings('ignore')
mlflow.set_tracking_uri("http://127.0.0.1:5000/")
mlflow.set_experiment("MLOpsClassificationTexte")
mlruns_dir = r"C:/Users/Abdessamad/Desktop/MLOpsClassificationTexteV2/mlruns"
custom_artifacts_dir = r"C:/Users/Abdessamad/Desktop/MLOpsClassificationTexteV2/mlartifacts"
os.makedirs(mlruns_dir, exist_ok=True)
os.makedirs(custom_artifacts_dir, exist_ok=True)

In [ ]:
X_test: List[str] = []
y_test: np.ndarray = np.array([])
# max_chunks = 3
# count = 0
for chunk in loader.generer_donnees_test():
    textes:List[str] = chunk["text"].astype(str).tolist()
    X_test.extend(textes)  # Liste plate
    y_test = np.append(y_test, np.array(chunk["label"]))
    # count += 1
    # # y_test = np.where(y_test <=2, 0, 1)
    # if count >= max_chunks:
    #     break

In [ ]:
def MLflow_1()->None:
    max_iters:List[int] = [random.randint(10, 100) for _ in range(1)]
    for max_iter in max_iters:
        #                               logistic
        with mlflow.start_run(run_name="logistic"):        
            mlflow.log_param("model", "logistic")
            mlflow.log_param("max_iter",max_iter)
            rapport_train,model,vectorizer=train.Model("logistic",max_iter)
            X_vect_test = vectorizer.transform(X_test)
            # print(rapport_train)
            mlflow.log_metric('accuracy_train', rapport_train['accuracy'])
            mlflow.log_metric('recall_class_1_train', rapport_train['1']['recall'])
            mlflow.log_metric('recall_class_0_train', rapport_train['0']['recall'])
            mlflow.log_metric('f1_score_macro_train', rapport_train['macro avg']['f1-score'])
            rapport_test=model.evaluer(X_vect_test,y_test)
            # print(rapport_test)
            mlflow.log_metric('accuracy_test', rapport_test['accuracy'])
            mlflow.log_metric('recall_class_1_test', rapport_test['1.0']['recall'])
            mlflow.log_metric('recall_class_0_test', rapport_test['0.0']['recall'])
            mlflow.log_metric('f1_score_macro_test', rapport_test['macro avg']['f1-score'])
        print("logistic est termines")
        #                               MLPClassifier
        with mlflow.start_run(run_name="MLPClassifier"):        
            mlflow.log_param("model", "MLPClassifier")
            mlflow.log_param("max_iter",max_iter)
            rapport_train,model,vectorizer=train.Model("MLPClassifier",max_iter=max_iter)
            mlflow.log_metric('accuracy_train', rapport_train['accuracy'])
            mlflow.log_metric('recall_class_1_train', rapport_train['1']['recall'])
            mlflow.log_metric('recall_class_0_train', rapport_train['0']['recall'])
            mlflow.log_metric('f1_score_macro_train', rapport_train['macro avg']['f1-score'])
            rapport_test=model.evaluer(X_vect_test,y_test)
            mlflow.log_metric('accuracy_test', rapport_test['accuracy'])
            mlflow.log_metric('recall_class_1_test', rapport_test['1.0']['recall'])
            mlflow.log_metric('recall_class_0_test', rapport_test['0.0']['recall'])
            mlflow.log_metric('f1_score_macro_test', rapport_test['macro avg']['f1-score'])
        print("MLPClassifier est termines")

# MLflow_1()


In [ ]:
def mlflow_2()->None:
    n_estimators: List[int]=[random.randint(50, 100) for _ in range(2)]
    learning_rates: List[float]=[0.1,0.2]
    max_depths: List[int]=[random.randint(1, 20) for _ in range(1)]
    # 'relu', 'logistic'
    for n in n_estimators:
        for max_depth in max_depths:
            with mlflow.start_run(run_name="RandomForestModel"):        
                mlflow.log_param("model", "RandomForestModel")
                mlflow.log_param("n_estimators",n)
                mlflow.log_param("max_depth",max_depth)
                rapport_train,model,vectorizer=train.Model("RandomForestModel",n_estimators=n,max_depth=max_depth)
                X_vect_test = vectorizer.transform(X_test)
                mlflow.log_metric('accuracy_train', rapport_train['accuracy'])
                mlflow.log_metric('recall_class_1_train', rapport_train['1']['recall'])
                mlflow.log_metric('recall_class_0_train', rapport_train['0']['recall'])
                mlflow.log_metric('f1_score_macro_train', rapport_train['macro avg']['f1-score'])
                rapport_test=model.evaluer(X_vect_test,y_test)
                mlflow.log_metric('accuracy_test', rapport_test['accuracy'])
                mlflow.log_metric('recall_class_1_test', rapport_test['1.0']['recall'])
                mlflow.log_metric('recall_class_0_test', rapport_test['0.0']['recall'])
                mlflow.log_metric('f1_score_macro_test', rapport_test['macro avg']['f1-score'])
            print("RandomForestModel est termines") 
            for learning_rate in learning_rates:
                with mlflow.start_run(run_name="GradientBoostingModel"):        
                    mlflow.log_param("model", "GradientBoostingModel")
                    mlflow.log_param("n_estimators",n)
                    mlflow.log_param("max_depth",max_depth)
                    mlflow.log_param("learning_rate",learning_rate)
                    rapport_train,model,vectorizer=train.Model("GradientBoostingModel",n_estimators=n,learning_rate=learning_rate,max_depth=max_depth)
                    mlflow.log_metric('accuracy_train', rapport_train['accuracy'])
                    mlflow.log_metric('recall_class_1_train', rapport_train['1']['recall'])
                    mlflow.log_metric('recall_class_0_train', rapport_train['0']['recall'])
                    mlflow.log_metric('f1_score_macro_train', rapport_train['macro avg']['f1-score'])
                    rapport_test=model.evaluer(X_vect_test,y_test)
                    mlflow.log_metric('accuracy_test', rapport_test['accuracy'])
                    mlflow.log_metric('recall_class_1_test', rapport_test['1.0']['recall'])
                    mlflow.log_metric('recall_class_0_test', rapport_test['0.0']['recall'])
                    mlflow.log_metric('f1_score_macro_test', rapport_test['macro avg']['f1-score'])
                print("GradientBoostingModel est termines") 

mlflow_2()

In [6]:
# with mlflow.start_run(run_name="MLPClassifier"):        
#     mlflow.log_param("model", "MLPClassifier")
#     mlflow.log_param("max_iter",100)
#     print("1")
#     rapport_train,model,vectorizer=train.Model("MLPClassifier",max_iter=100)
#     X_vect_test = vectorizer.transform(X_test)
#     print("1")
#     mlflow.log_metric('accuracy_train', rapport_train['accuracy'])
#     mlflow.log_metric('recall_class_1_train', rapport_train['1']['recall'])
#     mlflow.log_metric('recall_class_0_train', rapport_train['0']['recall'])
#     mlflow.log_metric('f1_score_macro_train', rapport_train['macro avg']['f1-score'])
#     print("1")
#     rapport_test=model.evaluer(X_vect_test,y_test)
#     print("1")
#     mlflow.log_metric('accuracy_test', rapport_test['accuracy'])
#     mlflow.log_metric('recall_class_1_test', rapport_test['1']['recall'])
#     mlflow.log_metric('recall_class_0_test', rapport_test['0']['recall'])
#     mlflow.log_metric('f1_score_macro_test', rapport_test['macro avg']['f1-score'])
# print("MLPClassifier est termines")  
# y_test=y_test.astype(int)
# @log_execution_time
# @log_function_call
# n_estimator=100
# learning_rate=0.1
# max_depth=6
# # 'relu', 'logistic'
# with mlflow.start_run(run_name="RandomForestModel"):        
#     mlflow.log_param("model", "RandomForestModel")
#     mlflow.log_param("n_estimators",n_estimator)
#     mlflow.log_param("max_depth",max_depth)
#     print("1")
#     rapport_train,model,vectorizer=train.Model("RandomForestModel",n_estimators=n_estimator,max_depth=max_depth)
#     X_vect_test = vectorizer.transform(X_test)
#     print(rapport_train)
#     print(rapport_train.keys())
#     mlflow.log_metric('accuracy_train', rapport_train['accuracy'])
#     mlflow.log_metric('recall_class_1_train', rapport_train['1']['recall'])
#     mlflow.log_metric('recall_class_0_train', rapport_train['0']['recall'])
#     mlflow.log_metric('f1_score_macro_train', rapport_train['macro avg']['f1-score'])
#     print("1")
#     rapport_test=model.evaluer(X_vect_test,y_test)
#     print(rapport_test)
#     mlflow.log_metric('accuracy_test', rapport_test['accuracy'])
#     mlflow.log_metric('recall_class_1_test', rapport_test['1']['recall'])
#     mlflow.log_metric('recall_class_0_test', rapport_test['0']['recall'])
#     mlflow.log_metric('f1_score_macro_test', rapport_test['macro avg']['f1-score'])
# print("RandomForestModel est termines") 

In [7]:
# with mlflow.start_run(run_name="MLPClassifier"):        
#     mlflow.log_param("model", "MLPClassifier")
#     mlflow.log_param("max_iter",100)
#     print("1")
#     rapport_train,model,vectorizer=train.Model("MLPClassifier",max_iter=100)
#     X_vect_test = vectorizer.transform(X_test)
#     print("1")
#     mlflow.log_metric('accuracy_train', rapport_train['accuracy'])
#     mlflow.log_metric('recall_class_1_train', rapport_train['1']['recall'])
#     mlflow.log_metric('recall_class_0_train', rapport_train['0']['recall'])
#     mlflow.log_metric('f1_score_macro_train', rapport_train['macro avg']['f1-score'])
#     print("1")
#     rapport_test=model.evaluer(X_vect_test,y_test)
#     print("1")
#     mlflow.log_metric('accuracy_test', rapport_test['accuracy'])
#     mlflow.log_metric('recall_class_1_test', rapport_test['1']['recall'])
#     mlflow.log_metric('recall_class_0_test', rapport_test['0']['recall'])
#     mlflow.log_metric('f1_score_macro_test', rapport_test['macro avg']['f1-score'])
# print("MLPClassifier est termines")  
# y_test=y_test.astype(int)
# @log_execution_time
# @log_function_call
# n_estimator=100
# learning_rate=0.1
# max_depth=6
# # 'relu', 'logistic'
# with mlflow.start_run(run_name="RandomForestModel"):        
#     mlflow.log_param("model", "RandomForestModel")
#     mlflow.log_param("n_estimators",n_estimator)
#     mlflow.log_param("max_depth",max_depth)
#     print("1")
#     rapport_train,model,vectorizer=train.Model("RandomForestModel",n_estimators=n_estimator,max_depth=max_depth)
#     X_vect_test = vectorizer.transform(X_test)
#     print("1")
#     mlflow.log_metric('accuracy_train', rapport_train['accuracy'])
#     mlflow.log_metric('recall_class_1_train', rapport_train['1']['recall'])
#     mlflow.log_metric('recall_class_0_train', rapport_train['0']['recall'])
#     mlflow.log_metric('f1_score_macro_train', rapport_train['macro avg']['f1-score'])
#     print("1")
#     rapport_test=model.evaluer(X_vect_test,y_test)
#     print("1")
#     mlflow.log_metric('accuracy_test', rapport_test['accuracy'])
#     mlflow.log_metric('recall_class_1_test', rapport_test['1']['recall'])
#     mlflow.log_metric('recall_class_0_test', rapport_test['0']['recall'])
#     mlflow.log_metric('f1_score_macro_test', rapport_test['macro avg']['f1-score'])
# print("RandomForestModel est termines") 

In [8]:
# with mlflow.start_run(run_name="GradientBoostingModel"):        
#     mlflow.log_param("model", "GradientBoostingModel")
#     mlflow.log_param("n_estimators",n_estimator)
#     mlflow.log_param("max_depth",max_depth)
#     mlflow.log_param("learning_rate",learning_rate)
#     rapport_train,model,vectorizer=train.Model("GradientBoostingModel",n_estimators=n_estimator,learning_rate=learning_rate,max_depth=max_depth)
#     mlflow.log_metric('accuracy_train', rapport_train['accuracy'])
#     mlflow.log_metric('recall_class_1_train', rapport_train['1']['recall'])
#     mlflow.log_metric('recall_class_0_train', rapport_train['0']['recall'])
#     mlflow.log_metric('f1_score_macro_train', rapport_train['macro avg']['f1-score'])
#     rapport_test=model.evaluer(X_vect_test,y_test)
#     mlflow.log_metric('accuracy_test', rapport_test['accuracy'])
#     mlflow.log_metric('recall_class_1_test', rapport_test['1.0']['recall'])
#     mlflow.log_metric('recall_class_0_test', rapport_test['0.0']['recall'])
#     mlflow.log_metric('f1_score_macro_test', rapport_test['macro avg']['f1-score'])
# print("GradientBoostingModel est termines") 

In [9]:
commentaires = [
    "Epic and emotional story.",
    "Terrible acting and bad plot.",
    "A masterpiece of modern cinema.",
    "Boring and too long.",
    "Amazing visual effects!"
]
def a():
    # Chargement du modèle, du vectorizer, etc.
    _, model, vectorizer = train.Model("logistic", 25)

    # Transformation du texte avec le vectorizer
    vect_data = vectorizer.transform(commentaires)

    # Prédiction
    predictions = model.predire(vect_data)

    results = ["Positive" if p == 1 else "Negative" for p in predictions]
    return results

print(a())


['Positive', 'Negative', 'Positive', 'Positive', 'Positive']
